In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

In [4]:
df = pd.read_csv('../data/zipcode_averages_nyc.csv')

In [5]:
df

,borough,year,11201,11203,11204,11205,11206,11207,11208,11209,...,10304,10305,10306,10307,10308,10309,10310,10311,10312,10314
0,borough,year,11201.00,11203.00,11204.00,11205.00,11206.00,11207.00,11208.00,11209.00,...,10304.00,10305.00,10306.00,10307.00,10308.00,10309.00,10310.00,10311.0,10312.00,10314.00
1,BROOKLYN,2005,1339839.71,541446.97,715778.03,540820.27,483532.79,482910.22,438755.87,528869.12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BROOKLYN,2006,1407938.22,594847.73,767228.76,699321.87,579856.22,540910.39,538224.18,709560.21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BROOKLYN,2007,1222110.87,735370.20,762102.32,722554.99,620309.62,595316.96,554003.47,568125.72,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BROOKLYN,2008,1781774.82,602419.45,668825.44,687707.63,852769.37,503992.83,516531.18,559853.18,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,STATEN ISLAND,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,589733.50,491369.88,473808.86,533005.66,425735.46,571864.08,426481.01,NaN,489051.72,441106.59
72,STATEN ISLAND,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,640458.19,464705.44,452044.51,612087.76,470180.28,692846.97,477136.71,NaN,523308.46,541750.45
73,STATEN ISLAND,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,566773.65,523853.75,506334.99,714094.41,543827.24,631891.77,476666.11,NaN,540472.73,638159.97
74,STATEN ISLAND,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,553238.31,711189.22,557266.35,696870.18,578466.32,757653.71,498948.30,NaN,607637.96,569120.81
